[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [3]:
# Read dataset with extra params sep='\t', encoding="latin-1"
sentiment = pd.read_csv("./labeledTrainData.tsv", sep="\t", encoding="latin-1")

sentiment.head(10)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
5,8196_8,1,I dont know why people think this is such a ba...
6,7166_2,0,"This movie could have been very good, but come..."
7,10633_1,0,I watched this video at a friend's house. I'm ...
8,319_1,0,"A friend of mine bought this film for Â£1, and..."
9,8713_10,1,<br /><br />This movie is full of references. ...


## 2. Preprocessing

In [4]:
# stop words
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lap12888/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [5]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [11]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()
def tokenizer_porter(text):
    token = []
    token.append(porter.stem(text))
    return token # Your code here

In [7]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = sentiment['review']
y = sentiment['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [12]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(stop_words=stop,
                       tokenizer=tokenizer_porter,
                       preprocessor=preprocessor)   # Your code here

clf = Pipeline([('vect', tfidf),
              ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/home/lap12888/anaconda3/envs/trongdinh/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a ', 'about ', 'above ', 'after ', 'again ', 'against ', 'ain ', 'all ', 'am ', 'an ', 'and ', 'any ', 'are ', 'aren ', 'aren t ', 'as ', 'at ', 'be ', 'because ', 'been ', 'before ', 'being ', 'below ', 'between ', 'both ', 'but ', 'by ', 'can ', 'couldn ', 'couldn t ', 'd ', 'did ', 'didn ', 'didn t ', 'do ', 'does ', 'doesn ', 'doesn t ', 'doing ', 'don ', 'don t ', 'down ', 'during ', 'each ', 'few ', 'for ', 'from ', 'further ', 'had ', 'hadn ', 'hadn t ', 'has ', 'hasn ', 'hasn t ', 'have ', 'haven ', 'haven t ', 'having ', 'he ', 'her ', 'here ', 'hers ', 'herself ', 'him ', 'himself ', 'his ', 'how ', 'i ', 'if ', 'in ', 'into ', 'is ', 'isn ', 'isn t ', 'it ', 'it s ', 'its ', 'itself ', 'just ', 'll ', 'm ', 'ma ', 'me ', 'mightn ', 'mightn t ', 'more '

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [21]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

0.49946666666666667
[[3734    0]
 [3754   12]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      3734
           1       1.00      0.00      0.01      3766

   micro avg       0.50      0.50      0.50      7500
   macro avg       0.75      0.50      0.34      7500
weighted avg       0.75      0.50      0.33      7500



## 5. Export Model 

In [9]:
# Using pickle to export our trained model
import pickle
import os